In [61]:
import os
import wfdb
import numpy as np
from scipy.signal import resample
from biosppy.signals import ecg
from sklearn.preprocessing import StandardScaler
import torch
import pandas as pd
from tqdm import tqdm


In [ ]:
def preprocess_ptb_record(record_path, target_fs=500, segment_len=300, pre_R=99):
    record = wfdb.rdrecord(record_path)
    signals = record.p_signal
    standard_leads = ['i', 'ii', 'iii', 'avr', 'avl', 'avf',
                      'v1', 'v2', 'v3', 'v4', 'v5', 'v6']
    lead_indices = [i for i, name in enumerate(record.sig_name) if name.lower() in standard_leads]
    signals = signals[:, lead_indices]
    original_fs = record.fs

    if original_fs != target_fs:
        n_samples = int(signals.shape[0] * target_fs / original_fs)
        signals = resample(signals, n_samples, axis=0)

    lead_index = 1 if signals.shape[1] > 1 else 0
    r_peaks = ecg.ecg(signal=signals[:, lead_index], sampling_rate=target_fs, show=False)['rpeaks']

    scaler = StandardScaler()
    signals = scaler.fit_transform(signals)

    segments = []
    for r in r_peaks:
        start = r - pre_R
        end = start + segment_len
        if start >= 0 and end <= signals.shape[0]:
            segment = signals[start:end, :].T  # (12, 300)
            segments.append(segment)

    return np.array(segments)  # shape = (n_beats, 12, 300)


def extract_label_from_hea(hea_path):
    label_mapping = {
    "inferior": "IMI",
    "antero-septal": "ASMI",
    "infero-lateral": "ILMI",
    "antero-lateral": "ALMI",
    "anterior": "AMI",
    "no": "NO",
    "infero-postero-lateral": "IPLMI",
    "postero-lateral": "PLMI",
    "posterior": "PMI",
    "lateral": "LMI",
    "infero-poster-lateral": "IPLMI",
    "infero-latera": "ILMI",  # typo fix
    "antero-septo-lateral": "ASLMI",
    "infero-posterior": "IPMI"
    }
    try:
        with open(hea_path, 'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "Diagnose" in line:
                    next_line = lines[i + 1].lower()
                    if "myocardial infarction" in next_line:
                        next_second_line = lines[i + 2].lower()
                        subtype_raw = next_second_line.replace("# acute infarction (localization): ", "").strip()
                        subtype_mapped = label_mapping.get(subtype_raw, subtype_raw.upper())
                        return "MI", subtype_mapped
                    elif "healthy" in next_line:
                        return "Healthy", 0
                    else:
                        return "Non_MI", 0
    except Exception as e:
        print(f"Không đọc được file .hea tại {hea_path}: {e}")
    return "Unknown"


def process_all_records(ptb_root='./ptbdb', output_dir='processed_ptb'):
    os.makedirs(output_dir, exist_ok=True)
    csv_entries = []

    for root, dirs, files in os.walk(ptb_root):
        dat_files = [f for f in files if f.endswith('.dat')]
        for dat_file in tqdm(dat_files):
            record_name = os.path.splitext(dat_file)[0]
            record_path = os.path.join(root, record_name)

            hea_file = os.path.join(root, record_name + '.hea')
            label, subtype = extract_label_from_hea(hea_file)

            try:
                segments = preprocess_ptb_record(record_path)
                print(segments.shape)
                if segments.size == 0:
                    continue

                patient_number = os.path.basename(os.path.dirname(record_path)).replace('patient', '')
                record_dir = os.path.join(output_dir, patient_number, record_name)
                os.makedirs(record_dir, exist_ok=True)

                for i, segment in enumerate(segments):
                    segment_filename = f"beat{i}.pt"
                    segment_path = os.path.join(record_dir, segment_filename)
                    torch.save(torch.tensor(segment, dtype=torch.float32), segment_path)

                    csv_entries.append({
                        'patient_number': patient_number,
                        'pt_path': segment_path.replace('\\', '/'),
                        'label': label, 
                        'subtype': subtype
                    })

            except Exception as e:
                print(f"Lỗi khi xử lý {record_path}: {e}")

    df = pd.DataFrame(csv_entries)
    csv_path = os.path.join(output_dir, 'ptb_heartbeat_paths.csv')
    df.to_csv(csv_path, index=False)
    print(f"Đã lưu danh sách nhịp tim và label tại: {csv_path}")



In [63]:
# import wfdb
# import matplotlib.pyplot as plt

# # 🔁 Thay bằng đường dẫn tới file .dat (không cần phần mở rộng)
# record_path = 'ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0/patient001/s0010_re'

# # Đọc toàn bộ tín hiệu ECG (kèm metadata)
# record = wfdb.rdrecord(record_path)

# # Lấy tín hiệu ECG (numpy array)
# signals = record.p_signal  # shape: (số mẫu, số chuyển đạo)
# lead_names = record.sig_name

# # 👀 In thông tin cơ bản
# print(f"Số mẫu: {signals.shape[0]}, Số chuyển đạo: {signals.shape[1]}")
# print(f"Tên các chuyển đạo: {lead_names}")

# # ✅ Vẽ 6 chuyển đạo đầu tiên (tuỳ chọn)
# plt.figure(figsize=(15, 10))

# for i in range(min(2, signals.shape[1])):
#     plt.subplot(2, 1, i + 1)
#     plt.plot(signals[:, i], linewidth=0.8)
#     plt.title(f'Lead: {lead_names[i]}')
#     plt.xlabel('Sample')
#     plt.ylabel('Amplitude')

# plt.tight_layout()
# plt.show()


In [64]:
# pt_path = 'processed_ptb/001/s0010_re/beat0.pt'

# # Đọc dữ liệu
# data = torch.load(pt_path)

# # Kiểm tra kích thước và nội dung
# print(data.shape)   # (số nhịp tim, 3600)
# print(data)      # Nhịp tim đầu tiên (đã flatten từ 12 leads)

In [68]:
df = pd.read_csv('processed_ptb/ptb_heartbeat_paths.csv')
df['subtype'].value_counts()

subtype
0        21418
IMI      12748
ASMI     11525
ILMI      8171
ALMI      6654
AMI       6417
IPLMI     2714
NO        2350
PLMI       776
PMI        466
LMI        461
ASLMI      274
IPMI        49
Name: count, dtype: int64

In [69]:
len(df['subtype'].unique())

13

In [ ]:
PTB_root = './ptb-diagnostic-ecg-database-1.0.0/ptb-diagnostic-ecg-database-1.0.0'
process_all_records(ptb_root=PTB_root)

0it [00:00, ?it/s]
 33%|███▎      | 1/3 [00:00<00:00,  3.87it/s]

(51, 12, 300)


 67%|██████▋   | 2/3 [00:00<00:00,  3.15it/s]

(165, 12, 300)


100%|██████████| 3/3 [00:01<00:00,  2.89it/s]


...

(137, 12, 300)


100%|██████████| 2/2 [00:00<00:00,  3.64it/s]


(144, 12, 300)


100%|██████████| 1/1 [00:00<00:00,  2.06it/s]

(149, 12, 300)
Đã lưu danh sách nhịp tim và label tại: processed_ptb\ptb_heartbeat_paths.csv
